In [160]:
import numpy as np
import math
from scipy.spatial.distance import cityblock
import matplotlib.pyplot as plt
from re import split
from field_info import *
from collections import Counter

In [135]:
class picker_state(object):
    def __init__(self, p_m_vel, p_w_vel, p_m_vel_est, p_w_vel_est, initial_p_pos, predict_model, center, picker_NO): 
        self.p_pos = initial_p_pos
        self._pos_nxt_x = initial_p_pos[0]+picker_num*x_offset
        self.p_m_vel = p_m_vel
        self.p_w_vel = p_w_vel
        self.p_m_vel_est = p_m_vel_est
        self.p_w_vel_est = p_w_vel_est
        self.pred_m = int(predict_model)
        self.ft_est = self.static_est() # initially guess from statistics data 
        self.ft_est_cur = self.ft_est
        self.picking_time=0 # record how long of picking
        self.center = center
        self.wait_time = 0 # this variable is updated from outside signal
        self.picker_NO = picker_NO
        # self.change_fur_t = _walk_t_nxt(initial_p_pos)
        self.walk_time = self._walk_t_nxt(initial_p_pos)
        self.walk = False
        self.serve_ready = False
        self.wait = False
        self.sample_ft()
        # self.cur_fur = _fur_cal()
    def mechanistic_est_update(self,delta_t):
        self.ft_est_cur = self.ft_one_tray - delta_t
        if self.ft_est_cur <= 0:
            self.ft_est_cur = 0
        return self.ft_est_cur
    def static_est(self): # initial guess of finishing time of one tray, it should be updated in bayesian model
        if self.pred_m == 0:
            mean, sigma = fast_mean_est, fast_sigma_est # global variable
        if self.pred_m == 1:
            mean, sigma = medi_mean_est, medi_sigma_est 
        if self.pred_m == 2:
            mean, sigma = slow_mean_est, slow_sigma_est 
        return mean, sigma
    # updating for simulation, true value but not accessible for estimation
    def sample_ft(self):
        sample=-1
        if self.pred_m == 0:
            mean = fast_mean
            sigma = fast_sigma # global variable
        if self.pred_m == 1:
            mean = medi_mean
            sigma = medi_sigma 
        if self.pred_m == 2:
            mean = slow_mean
            sigma = slow_sigma
        while(sample >= mean+sigma or sample <= mean-sigma):
            sample = np.random.normal(mean, sigma) # sampling from a unknown distribution as a ground truth
        self.ft_tru_cur=sample
        self.picking_time=0
        self.wait = False
    def _walk_t_nxt(self, p_pos):
        pos_start = np.asarray([p_pos[0],y_head])
        pos_end = np.asarray([p_pos[0]+x_offset*picker_num, y_offset])
        t = cityblock(pos_start,pos_end)/self.p_w_vel
        return t
    def state_update(self, delta_t): 
        if self.ft_tru_cur > 0:
            if not self.walk:
                ft_cur_fur = (self.p_pos[1]-y_head)/self.p_m_vel
                if self.ft_tru_cur >= ft_cur_fur: 
                    self.ft_tru_cur += self.walk_time # need to change furrow to finsih a tray
                    if delta_t <= ft_cur_fur:
                        self.ft_tru_cur -= delta_t
                        self.p_pos[1] -= delta_t*self.p_m_vel
                        self.picking_time += delta_t
                    elif delta_t >= self.ft_tru_cur:
                        self.p_pos[0] = self._pos_nxt_x
                        d_t = self.ft_tru_cur - self.walk_time - ft_cur_fur 
                        self.p_pos[1] = y_offset - d_t*p_m_vel
                        # if not self.serve_ready:
                        #     self.wait_time += delta_t - self.ft_tru_cur
                        #     # print "here!"
                        # self.ft_tru_cur = 0
                        # self.walk_time = _walk_t_nxt(self.p_pos)
                        # self._pos_nxt_x = self.p_pos[0]+x_offset*picker_num
                        # self.walk = False
                        # self.center += picker_num*x_offset
                        if not self.serve_ready:
                            self.wait_time += delta_t - self.ft_tru_cur
                            self.ft_tru_cur = 0  
                            self.picking_time += self.ft_tru_cur-self.walk_time
                            self.wait = True
                        else:
                            self.sample_ft()  
                            delta_t -= self.ft_tru_cur
                            self.serve_ready=False
                            # self.picking_time = 0
                            self.state_update(delta_t)
                            # print "here!"
                        self._pos_nxt_x = self.p_pos[0]+x_offset*picker_num
                        self.walk = False
                        self.center += picker_num*x_offset
                    else: # walk flag means that picker will keep walking state during delta_t
                        self.walk = True
                        # update to the instant at y_head
                        self.ft_tru_cur -= ft_cur_fur
                        delta_t -= ft_cur_fur
                        self.picking_time += ft_cur_fur
                else: # picker finish the tray in current furrow
                    self.walk = False
                    if self.ft_tru_cur >= delta_t:
                        self.p_pos[1] -= delta_t*self.p_m_vel
                        self.ft_tru_cur -= delta_t
                        self.picking_time += delta_t
                    else:
                        self.p_pos[1] -= self.ft_tru_cur*self.p_m_vel
                        if not self.serve_ready:
                            # print "here"
                            # print delta_t, self.ft_tru_cur
                            self.wait_time += delta_t-self.ft_tru_cur
                            self.picking_time += self.ft_tru_cur
                            self.ft_tru_cur = 0
                            self.wait = True
                        else:
                            self.sample_ft()
                            delta_t -= self.ft_tru_cur
                            # self.picking_time = 0
                            self.serve_ready=False
                            self.state_update(delta_t)
            if self.walk:
                if delta_t <= self.walk_time:
                    self.walk_time -= delta_t
                    self.ft_tru_cur -= delta_t
                    if delta_t*self.p_w_vel <= self._pos_nxt_x-self.p_pos[0]:
                        p_pos[0] += delta_t*self.p_w_vel # means that pickers are walking
                        p_pos[1] = y_head
                    else:
                        self.p_pos[0] = self._pos_nxt_x
                        t_x = (self._pos_nxt_x - self.p_pos[0])/self.p_w_vel
                        p_pos[1] =  y_head + (delta_t-t_x)*self.p_w_vel
                elif delta_t < self.ft_tru_cur:
                    self.ft_tru_cur -= delta_t
                    self.picking_time += delta_t-self.walk_time
                    self.p_pos[0] = self._pos_nxt_x
                    self.p_pos[1] = y_offset - self.p_m_vel*(delta_t-self.walk_time)
                else:
                    self.p_pos[0] = self._pos_nxt_x
                    self.p_pos[1] = y_offset - self.ft_tru_cur*self.p_m_vel
                    if not self.serve_ready:
                            self.wait_time += delta_t - self.ft_tru_cur
                            self.picking_time += self.ft_tru_cur-self.walk_time
                            self.ft_tru_cur = 0
                            self.wait=0
                    else:
                        self.sample_ft()
                        delta_t -= self.ft_tru_cur
                        self.serve_ready = False
                        # self.picking_time = 0
                        self.state_update(delta_t)
                    self.walk = False
                    self.center += picker_num*x_offset
                    # print "here!!!"
                    self._pos_nxt_x = self.p_pos[0]+x_offset*picker_num
        elif not self.serve_ready:
            self.wait_time += delta_t
            self.wait = True
            # print "here!!"
        else: # ready to serve and ft=0
            self.serve_ready = False
            self.sample_ft()
            # self.picking_time=0
            self.state_update(delta_t)

In [136]:
class robot_state(object):
    def __init__(self, r_vel):
        self.vel = r_vel
        self.exe_time = 0 # if 0, means finish task and start running back
        self.tar_pos = np.zeros(2) 
        self.p_NO = -1 # if -1 means, idle, otherwise serving picking NO.
        self.run_time = 0 # if 0 means arrival
        self.back_time = 0 # if 0 means idle
    def cal_time(self, ft_cur_tray, center):
        self.run_time = cityblock(self.tar_pos, center)/self.vel
        self.exe_time = np.maximum(ft_cur_tray, self.run_time)+PROC_TIME
        self.back_time = self.run_time + self.exe_time        

In [137]:
# initialize robots and pickers positions
def field_initialization():
    pickers = [] # for each pickers
    robots = []
    # model for each picker, ground truth and estimation
    # 0 means fast, 1 means medium, 2 means slow
    model_num = (np.zeros(N_pickers_fast),np.ones(N_pickers_slow)*2,np.ones(N_pickers_medi))
    model_num = np.concatenate(model_num).astype(int)   
    v_pickers = (np.ones(N_pickers_fast)*v_fast,np.ones(N_pickers_slow)*v_slow,np.ones(N_pickers_medi)*v_medi)
    v_pickers = np.concatenate(v_pickers)
    w_pickers = (np.ones(N_pickers_fast)*w_fast,np.ones(N_pickers_slow)*w_slow,np.ones(N_pickers_medi)*w_medi)
    w_pickers = np.concatenate(w_pickers)
    v_pickers_est = (np.ones(N_pickers_fast)*v_fast_est, np.ones(N_pickers_slow)*v_slow_est, np.ones(N_pickers_medi)*v_medi_est)
    v_pickers_est = np.concatenate(v_pickers_est)
    w_pickers_est = (np.ones(N_pickers_fast)*w_fast_est, np.ones(N_pickers_slow)*w_slow_est, np.ones(N_pickers_medi)*w_medi_est)
    w_pickers_est = np.concatenate(w_pickers_est)
    perm = np.random.permutation(picker_num)
    
    model_num = model_num[perm]
    v_pickers = v_pickers[perm]
    w_pickers = w_pickers[perm]
    v_pickers_est = v_pickers_est[perm]
    w_pickers_est = w_pickers_est[perm]
    # initialization for pickers position
    for i in range(picker_num):
        x = furrow_width/2 + (furrow_width+berry_width)*i
        pos = [x, y_head + furrow_length] # a list [x,y]
        picker = picker_state(v_pickers[i],w_pickers[i], v_pickers_est[i], w_pickers_est[i], pos, model_num[i], center_1, i)
        pickers.append(picker)
    
    for j in range(robot_num):
        robot = robot_state(robot_v)
        robots.append(robot)

    return pickers, robots

In [138]:
# est_ft_tray_n=[]
# p_pos_n2=[]
# est_ft_cur_n=[]
# est_v_pickers_n=[]
# est_w_pickers_n=[]
# wait_time_n=[]
# center_n=[]
# pickers, robots = field_initialization()    
# for picker in pickers:
#     p_pos_n2.append(picker.p_pos)
#     est_mean, est_sigma = picker.static_est()
#     est_ft_tray=0
#     while(est_ft_tray >= est_mean+est_sigma or est_ft_tray <= est_mean-est_sigma):
#         est_ft_tray = np.random.normal(est_mean,est_sigma)
#     est_ft_cur = est_ft_tray-picker.picking_time 
#     if est_ft_cur<0 or picker.wait: est_ft_cur = 0  # assuming that picker has done
#     est_ft_tray_n.append(est_ft_tray)
#     est_ft_cur_n.append(est_ft_cur)
#     est_v_pickers_n.append(picker.p_m_vel_est)
#     est_w_pickers_n.append(picker.p_w_vel_est)
#     wait_time_n.append(picker.wait_time)
#     center_n.append(picker.center)
# est_ft_cur_n = np.asarray(est_ft_cur_n)
# est_ft_tray_n = np.asarray(est_ft_tray_n)
# p_pos_n2 = np.asarray(p_pos_n2)
# center_n = np.asarray(center_n)
# r_run_time_m=[]
# r_back_time_m = []
# r_assign_m = []
# for robot in robots:
#     r_run_time_m.append(robot.run_time)
#     r_back_time_m.append(robot.back_time)
#     r_assign_m.append(robot.p_NO)
# print p_pos_n2
# print est_ft_tray_n
# print est_ft_cur_n
def sample_SCENE(sample_ft, p_pos, v_pickers, w_pickers, current_ft, pred_horizon,centers):
    scene={} # target position and finish interval
    x_offset = furrow_width+berry_width
    y_offset = y_head+furrow_length
    sample_nums=[]
    # The picker will keep their order in next pick_num furrows
    for i,ft_interval, ft_cur in zip(range(len(sample_ft)),sample_ft, current_ft):
        n = int(math.floor((pred_horizon-ft_cur)/ft_interval) + 1) # obtain sample points on a furrow
        sample_nums.append(n)
        pos_list, ft_list, center_list = [], [], []
        cur_pos = np.copy(p_pos[i].reshape(-1,1))
        cur_center = np.copy(centers[i]).reshape(-1,1)
        nxt_pos = np.copy(cur_pos)
        nxt_center = np.copy(cur_center)
        for j in range(n):
            if j==0:
                nxt_pos[1] = cur_pos[1] - v_pickers[i]*ft_cur # y
            else:
                nxt_pos[1] = cur_pos[1] - v_pickers[i]*ft_interval # y
            ft_tray = ft_interval # finish time of next tray
            # furrow change happened
            if nxt_pos[1] < y_head:
                nxt_pos[0] += picker_num*x_offset # keep the same order
                nxt_pos[1] = y_offset - (y_head-nxt_pos[1]) 
                fsat_tray = ft_interval + (cityblock(nxt_pos,cur_pos))/w_pickers[i] # finishing time of current tray should add walking time of picker
                nxt_center[0]+=picker_num*x_offset
            pos_list.append(nxt_pos)
            ft_list.append(ft_tray)
            center_list.append(nxt_center)
            cur_pos=np.copy(nxt_pos)
            cur_center=np.copy(nxt_center)
        if len(pos_list)>1:
            scene['picker'+str(i)]=np.concatenate(pos_list,axis=1).T
            scene['center'+str(i)]=np.concatenate(center_list,axis=1).T
        else:
            scene['picker'+str(i)]=nxt_pos.T
            scene['center'+str(i)]=nxt_center.T
        scene['ft'+str(i)]=np.asarray(ft_list) # also add 
    scene['sample_nums']=np.asarray(sample_nums)
    return scene
# scenario=sample_SCENE(est_ft_tray_n, p_pos_n2, est_v_pickers_n, est_w_pickers_n,est_ft_cur_n,1200, center_n)
# scenario

In [147]:
sample_nums=scenario['sample_nums']
print sample_nums
# def produce_perm(scene_sample):
#     perm=[]
#     for i in scene_sample.keys():
#         char_sp = split('(\d+)', i)
#         if char_sp[0]=='picker':
#             n=scene_sample[i].shape[0]
#             num_list=[]
#             for j in range(n):
#                 num_list.append(int(char_sp[1])+j*picker_num)
#             print num_list
#             perm.append(num_list)
#     perm=np.concatenate(perm)
#     perm=np.sort(perm)
#     return perm
def produce_perm(sample_nums):
    perm=[]
    for i in range(picker_num):
        for j in range(sample_nums[i]):
            perm.append(i+j*picker_num)
    perm=np.sort(perm)
    return perm
# perm = produce_perm(scenario)
perm = produce_perm(sample_nums)
print perm
# print scenario

[2 2 1 1 1 1 2 1 1 2 1 1]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 18 21]


In [148]:
def est_wait_time(scene_sample, robots, perm_a, time_w):
    w_t = np.copy(time_w)
    ft_cur = [scene_sample['ft'+str(i)][0] for i in range(picker_num)]  
    fur_orders = [0]*picker_num
    for i in perm_a:
        # Next idle robot
        robot_assign_m=[robot.p_NO for robot in robots]
        n=robot_assign_m.count(-1)
        if n>0:
            j=robot_assign_m.index(-1)
            picker_NO=i%picker_num
            point_order=int(i)/picker_num
            robots[j].tar_pos=scene_sample['picker'+str(picker_NO)][point_order,:]
            robots[j].p_NO=picker_NO
            robots[j].cal_time(scene_sample['ft'+str(picker_NO)][point_order],
                               scene_sample['center'+str(picker_NO)][point_order,:])
            continue
        else:
            
            r_back_time=[robot.back_time for robot in robots]
            r_serve_pickers=[robot.p_NO for robot in robots]
            r_not_serve=[p for p in range(picker_num) if p not in r_serve_pickers]
#             print "current assignment", robot_assign_m
#             print "current finish time", ft_cur
#             print "run back time", r_back_time
            delta_t = np.amin(r_back_time)
            idx_idle = np.argmin(r_back_time)
            for k in r_not_serve:
                if delta_t > ft_cur[k]:
                    ft_cur[k] = 0
                    w_t[k] += (delta_t-ft_cur[k])
                else:
                    ft_cur[k] -= delta_t
            for robot in robots:
                robot.back_time -= delta_t
                if robot.run_time >= delta_t:
                    robot.run_time -=delta_t
                    if ft_cur[robot.p_NO] >= delta_t:
                        ft_cur[robot.p_NO]-=delta_t
                    else:
                        w_t[robot.p_NO] += delta_t-ft_cur[robot.p_NO]
                        ft_cur[robot.p_NO]=0
                else:
                    if ft_cur[robot.p_NO] >= delta_t:
                        ft_cur[robot.p_NO]-=delta_t
                    else:
                        if ft_cur[robot.p_NO] < robot.run_time:
                            w_t[robot.p_NO] += robot.run_time-ft_cur[robot.p_NO]
                        fur_orders[robot.p_NO]+=1
                        order=fur_orders[robot.p_NO]
                        total_orders = len(scene_sample['ft'+str(robot.p_NO)])
                        if order > total_orders-1: # next point is not in the list
                            continue
                        else:
                            ft_cur[robot.p_NO]=scene_sample['ft'+str(robot.p_NO)][order]
                    robot.run_time = 0
            robots[idx_idle].p_NO=-1
    return w_t
# print len(robots)

In [149]:
perm=np.sort(perm)
idx_perm=np.random.permutation(range(len(perm)))
perm=perm[idx_perm]
print perm
sample_nums=scenario['sample_nums']
print sample_nums
# put constraint in the selected perm
def constraint(perm,sample_nums):
    constrain_perm=list(perm)
    for i in range(picker_num):
        fur_perm=[]
        fur_perm_idx=[]
        # obtain all furrow sample points
        if sample_nums[i]>1:
            for j in range(sample_nums[i]):
                perm_el = i+j*picker_num
                fur_perm.append(perm_el)
                fur_perm_idx.append(constrain_perm.index(perm_el))# index of furrow point in perm
            for j,perm_idx in enumerate(np.sort(fur_perm_idx)):
                constrain_perm[perm_idx]=fur_perm[j]
    return constrain_perm
constrain_perm=constraint(perm,sample_nums)
print constrain_perm

[ 0  1  3  8 21 18 12  4  2  7  5 11 13  9  6 10]
[2 2 1 1 1 1 2 1 1 2 1 1]
[0, 1, 3, 8, 9, 6, 12, 4, 2, 7, 5, 11, 13, 21, 18, 10]


In [125]:
# robots is a copy of input from real state!
robots=[robot_state(1) for i in range(robot_num)]
robot_assign_m=[robot.p_NO for robot in robots]
print robot_assign_m
robots_copy=list(robots)
min_cost=999999
min_perm=[]
# random method      
for i in range(1000):
    perm=list(np.random.permutation(perm))           
    perm=constraint(perm,sample_nums)
#     print perm
    w_t=est_wait_time(scenario, robots_copy, perm, wait_time_n)
    cost=np.mean(w_t)*0.8+np.std(w_t)*0.2
#     print cost
    if cost < min_cost:
        min_cost=cost
        min_perm=perm
print min_cost
print min_perm

[-1, -1, -1, -1]
276.722406647
[5, 9, 3, 4, 10, 0, 12, 1, 21, 11, 6, 18, 7, 8, 2, 13]


In [158]:
def cost_func(scenario, robots, perm, wait_time_n):
    w_t=est_wait_time(scenario, robots, perm, wait_time_n)
    return np.mean(w_t)*0.8+np.std(w_t)*0.2
def meta_heuristic_policy(N_search_times, perm, scenario, robots, wait_time_n):
    min_perm=np.copy(perm)
    sample_nums=scenario['sample_nums']
    min_cost = 999999
    for i in range(N_search_times):
        perm_trial=np.copy(min_perm)
        a=np.random.randint(len(perm_trial))
        b=np.random.randint(len(perm_trial))
        if a>b:
            a,b=b,a
        perm_trial[a:b+1]=perm_trial[a:b+1][::-1]
        perm_trial=constraint(perm_trial,sample_nums)
        trial_cost=cost_func(scenario, robots, perm_trial, wait_time_n)
        if trial_cost < min_cost:
            min_perm=np.copy(perm_trial)
            min_cost=trial_cost
            print min_perm
            print min_cost
    return min_perm
#     return np.mean(w_t)
# meta_heuristic
# robots=[robot_state(1) for i in range(robot_num)]

# for i in range(100000):
#     perm_trial=np.copy(min_perm)
#     a=np.random.randint(len(perm_trial))
#     b=np.random.randint(len(perm_trial))
#     if a>b:
#         a,b=b,a
#     perm_trial[a:b+1]=perm_trial[a:b+1][::-1]
#     perm_trial=constraint(perm_trial,sample_nums)
#     trial_cost=cost_func(scenario, robots, perm_trial, wait_time_n)
#     if trial_cost < min_cost:
#         min_perm=np.copy(perm_trial)
#         min_cost=trial_cost
#         print min_perm
#         print min_cost

In [162]:
def MSA_Policy(pickers, robots, s_nums):
    est_ft_tray_n=[]
    p_pos_n2=[]
    est_ft_cur_n=[]
    est_v_pickers_n=[]
    est_w_pickers_n=[]
    wait_time_n=[]
    center_n=[]
    pickers, robots = field_initialization()    
    for picker in pickers:
        p_pos_n2.append(picker.p_pos)
        est_mean, est_sigma = picker.static_est()
        est_ft_tray=0
        while(est_ft_tray >= est_mean+est_sigma or est_ft_tray <= est_mean-est_sigma):
            est_ft_tray = np.random.normal(est_mean,est_sigma)
        est_ft_cur = est_ft_tray-picker.picking_time 
        if est_ft_cur<0 or picker.wait: est_ft_cur = 0  # assuming that picker has done
        est_ft_tray_n.append(est_ft_tray)
        est_ft_cur_n.append(est_ft_cur)
        est_v_pickers_n.append(picker.p_m_vel_est)
        est_w_pickers_n.append(picker.p_w_vel_est)
        wait_time_n.append(picker.wait_time)
        center_n.append(picker.center)
    est_ft_cur_n = np.asarray(est_ft_cur_n)
    est_ft_tray_n = np.asarray(est_ft_tray_n)
    p_pos_n2 = np.asarray(p_pos_n2)
    center_n = np.asarray(center_n)
    first_selections = []
    N_search_times=10000
    pred_horizon = 1200
    # sampling scenario
    for itr in range(s_nums):
        print("*******sample %i********"%itr)
        scenario = sample_SCENE(est_ft_tray_n, p_pos_n2, est_v_pickers_n, 
                                est_w_pickers_n, est_ft_cur_n, pred_horizon, center_n)
        perm = produce_perm(scenario['sample_nums'])
        perm = np.random.permutation(perm)
        min_perm = meta_heuristic_policy(N_search_times, perm, scenario, robots, wait_time_n)
        first_selections.append(min_perm[0])
        print min_perm
    print first_selections
    count = Counter(first_selections)
    print count
    picker_NO=count.most_common()[0][0]
    tar_pos=scenario['picker'+str(picker_NO)][0]
    print tar_pos
    return tar_pos, picker_NO
MSA_Policy(pickers, robots, 10)

*******sample 0********
[ 4  9  6 11  1 10  5  0 18  8  2  3 16  7 17 19]
729.424381939
[ 4  9  6 11  1 10  5  0 18  8  2 16  3  7 17 19]
638.336751528
[ 4  9  6 11  1 10  5  0  3 16  2  8 18  7 17 19]
584.593562233
[ 4  9  6 18  8  2 16  3  0  5 10  1 11  7 17 19]
487.471700081
[ 4  2  8  6 18  9 16  3  0  5 10  1 11  7 17 19]
480.531013908
[ 4  2  8  6  5  7 11  1 10 17  0  3 16  9 18 19]
428.328110021
[ 4  2  8  6  5  7 19 18  9 16  3  0 17 10  1 11]
410.870955582
[ 4 10  5  0  3 16  9  6  7 19 17 18  8  2  1 11]
350.815152177
[ 4  5 10  0  3 16  9  6  7 19 17 18  8  2  1 11]
350.701898872
[ 4  5 10  0  3 16  9  6  8 18 17  7 19  2  1 11]
293.27764531
[ 4  5  7 19 17  6  8 18  9 16  3  0 10  2  1 11]
271.111918093
[ 4  5  7 19 17  6  8 18  9 16  3  0 10  2  1 11]
250.304506746
[ 4  5  7 19 17  6  8 18  9 16  3  0 10  2  1 11]
*******sample 1********
[ 7  5  4 10  9  6  2  0 17  3 18  8 19 11 16  1]
626.843441075
[ 3  5  0  2  6  9 10  4 17  7 18  8 19 11 16  1]
613.718925733
[ 3  5 

(array([  4.6228    ,  83.12120709]), 4)

In [75]:
def MSA_policy_Manual(pickers, robots): 
    p_NO = int(raw_input())
    tar_pos = np.zeros(2)
    # print pickers[p_NO].p_m_vel
    # print pickers[p_NO].ft_tru_cur
    tar_pos[1] = pickers[p_NO].p_pos[1]-pickers[p_NO].ft_tru_cur*pickers[p_NO].p_m_vel
    tar_pos[0] = pickers[p_NO].p_pos[0]
    if tar_pos[1]<y_head:
        t_f_fur=(pickers[p_NO].p_pos[1]-y_head)/pickers[p_NO].p_m_vel
        tar_pos[1]=y_offset-(pickers[p_NO].ft_tru_cur-t_f_fur)*pickers[p_NO].p_m_vel
        tar_pos[0]+=picker_num*x_offset
    print tar_pos
    # robot_serving = [robot.p_NO for robot in robots]
    # print robot_serving
    return tar_pos, p_NO

In [15]:
def state_print(pickers):
    ft=[]
    wt=[]
    for picker in pickers:
        ft.append(picker.ft_tru_cur)
        wt.append(picker.wait_time)
    print "waiting time",wt
    print "finishing time",ft

In [16]:
def main():
    pickers, robots = field_initialization()    
    ft_tray = [picker.ft_tru_cur for picker in pickers]
    model = [picker.pred_m for picker in pickers]
    print ft_tray, model
    total_time = 4*60*60 # every 3 hours, get a rest
    time = 0
    while time < total_time:
        r_assign = [robot.p_NO for robot in robots]
        # print "current assignment", r_assign
        n = r_assign.count(-1)
        if n > 1: # only for initialization with robot_num
            for i in range(n):
                tar_pos, p_NO = MSA_policy(pickers, robots) # assign means pickers' number
                robots[i].tar_pos = np.copy(tar_pos)
                robots[i].p_NO = p_NO
                robots[i].cal_time(pickers[robots[i].p_NO].ft_tru_cur,pickers[robots[i].p_NO].center) # calculate exe time, running time and back time
                print pickers[robots[i].p_NO].ft_tru_cur
            print [robot.back_time for robot in robots]

        elif n==1: # when all robot are assigned, we need to calculate delta_t of next event coming
             state_print(pickers)
             tar_pos, p_NO = MSA_policy(pickers, robots)
             idx_idle = r_assign.index(-1)
             robots[idx_idle].p_NO = p_NO
             robots[idx_idle].tar_pos = np.copy(tar_pos)
             robots[idx_idle].cal_time(pickers[robots[idx_idle].p_NO].ft_tru_cur,pickers[robots[idx_idle].p_NO].center)
        else: # update state to the instant of next robot back
            r_back_time=[robot.back_time for robot in robots]
            r_serve_pickers=[robot.p_NO for robot in robots]
            # r_exe_time=[robot.exe_time for robot in robots]
            print r_back_time
            # print r_serve_pickers
            r_not_serve=[p for p in range(picker_num) if p not in r_serve_pickers]
            # print r_not_serve
            delta_t = np.amin(r_back_time) # we want the system update to this instant
            print "passed time", delta_t
            time += delta_t
            idx_min = np.argmin(r_back_time)
            for i in r_not_serve:
                # print pickers[i].ft_tru_cur
                pickers[i].state_update(delta_t)
            for robot in robots:
                robot.back_time -= delta_t
                if robot.run_time > delta_t:
                    robot.run_time -= delta_t
                    pickers[robot.p_NO].state_update(delta_t)
                else:
                    pickers[robot.p_NO].serve_ready = True
                    robot.r_run_time = 0
                    pickers[robot.p_NO].state_update(delta_t)
                    # if robot.exe_time > delta_t: # check if picker finish their picking, if not
                    #     robot.exe_time -= delta_t
                    #     pickers[robot.p_NO].state_update(delta_t)
                    # else: # if yes, finishing current tray
                    #     pickers[robot.p_NO].state_update(robot.exe_time)
                    #     pickers[robot.p_NO].serve_ready=False
                    #     pickers[robot.p_NO].sample_ft()
                    #     pickers[robot.p_NO].state_update(delta_t-robot.exe_time)
            # print [pickers[robot.p_NO].serve_ready for robot in robots]
            r_assign = [robot.p_NO for robot in robots]
            print "current assignment", r_assign
            robots[idx_min].p_NO=-1

In [18]:
main()

[995.8365527795692, 1002.266018926611, 467.71397078940305, 471.3154793188159, 460.77791246108904, 856.0839649285001, 454.9226250834242, 640.6892600957159, 940.70749448287, 703.6921793526868, 633.3028930284262, 663.560098285674] [2, 2, 0, 0, 0, 2, 0, 1, 2, 1, 1, 1]
1
[   1.4224      104.39963516]
1002.26601893
2
[   2.4892      111.70338925]
467.713970789
3
[   3.556       111.49954387]
471.315479319
4
[   4.6228      112.09597015]
460.777912461
[1121.6440540887843, 593.32896004272288, 595.65982318937097, 584.65188261579135]
[1121.6440540887843, 593.32896004272288, 595.65982318937097, 584.65188261579135]
passed time 584.651882616
current assignment [1, 2, 3, 4]
waiting time [0, 0, 0, 0, 0, 0, 129.72925753236717, 0, 0, 0, 0, 0]
finishing time [411.18467016377781, 417.61413631081962, 332.40971103189747, 320.88663512514802, 336.30840241561486, 271.43208231270876, 0, 56.037377479924544, 356.0556118670786, 119.04029673689547, 48.651010412634832, 78.908215669882679]
7
[   7.8232      111.0107

4
[  4.6228      86.03279409]
[58.077285067070591, 28.711196220453218, 185.62158817662709, 557.03896051340189]
passed time 28.7111962205
current assignment [3, 2, 4, 7]
waiting time [125.80750130921518, 0, 204.58246044109552, 221.56667154081146, 773.40823154405086, 496.88310550294125, 898.04444534801735, 0, 0, 649.27489107875454, 201.48737067967153, 0]
finishing time [396.14709563669078, 1264.4680538012744, 840.36718683552283, 365.10773548827399, 0, 605.13228321312079, 159.38035169556724, 438.48305396484682, 791.94655908472703, 413.26229040016995, 1166.2948363794308, 150.84713025158169]
11
[ 12.0904      83.55046837]
[29.366088846617373, 250.08719862315053, 156.91039195617387, 528.3277642929487]
passed time 29.3660888466
current assignment [3, 11, 4, 7]
waiting time [125.80750130921518, 0, 204.58246044109552, 221.56667154081146, 802.77432039066821, 496.88310550294125, 898.04444534801735, 0, 0, 649.27489107875454, 201.48737067967153, 0]
finishing time [366.78100679007343, 1235.101964954

KeyboardInterrupt: 